In [1]:
import os
os.environ['KMP_AFFINITY'] = 'noverbose'
os.environ['KMP_WARNINGS'] = '0'
os.environ['KMP_SETTINGS'] = '1'

import Metashape

# create conn to the database
import src.base.connect_to_database as ctd
conn = ctd.establish_connection()


User settings:

   KMP_AFFINITY=noverbose
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=200
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_MONOTONIC_DYNAMIC_SCHEDULE=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NESTING_MODE=0
 

In [2]:
PATH_PROJECT_FOLDERS = "/data_1/ATM/data_1/sfm/agi_projects"
project_name = "test2"

In [3]:
import src.base.load_credentials as lc

# get the license key
licence_key = lc.load_credentials("agisoft")['licence']

# Activate the license
Metashape.License().activate(licence_key)


License activated


In [4]:
image_ids = ["CA196532V0010", "CA196532V0011"]

# get only the first 3 images
#image_ids = image_ids[:3]

path_image_folder = "/data_1/ATM/data_1/aerial/TMA/downloaded"

# create lst with absolute paths
images_paths = [os.path.join(path_image_folder, image + ".tif") for image in image_ids]

# create sql list string for image_ids
image_ids_string = "','".join(image_ids)
image_ids_string = "('" + image_ids_string + "')"

# create a dict with the focal lengths
sql_string = f"SELECT image_id, focal_length FROM images_extracted WHERE image_id in {image_ids_string}"
focal_length_data = ctd.execute_sql(sql_string, conn)
focal_lengths = focal_length_data.set_index('image_id')['focal_length'].to_dict()

camera_positions = {}
camera_accuracies = {}

In [5]:
import src.load.load_image as li
import src.display.display_images as di

"""
images = []
for image_path in images_paths:
    image = li.load_image(image_path)
    images.append(image)

di.display_images(images)
"""

'\nimages = []\nfor image_path in images_paths:\n    image = li.load_image(image_path)\n    images.append(image)\n\ndi.display_images(images)\n'

In [6]:
import os
# create path to the project
project_fld = os.path.join(PATH_PROJECT_FOLDERS, project_name)

# remove the project folder if it exists
if os.path.exists(project_fld):
    import shutil
    shutil.rmtree(project_fld)

# create the project folder
os.makedirs(project_fld)



project_file_path = project_fld + "/" +  project_name + ".psx"

# create a new metashape project
doc = Metashape.Document()

# save the project
#doc.save(project_file_path)

In [7]:
# add a chunk
chunk = doc.addChunk()

# set the coordinate system of the chunk
chunk.crs = Metashape.CoordinateSystem("EPSG::3031")

# add the images to the chunk
chunk.addPhotos(images_paths)

AddPhotos


In [8]:
# Set some settings for each camera
for camera in chunk.cameras:

    # set to film camera
    camera.sensor.film_camera = True

    # set focal length if given
    if camera.label in focal_lengths:
        focal_length = focal_lengths[camera.label]
        camera.sensor.focal_length = focal_length

    # set camera position if given
    if camera.label in camera_positions:
        x, y, z = camera_positions[camera.label]
        camera.reference.location = Metashape.Vector([x, y, z])

        # set the accuracy of the position if given
        if camera.label in camera_accuracies:
            accuracy = camera_accuracies[camera.label]
            camera.reference.accuracy = Metashape.Vector([accuracy[0], accuracy[1], accuracy[2]])

In [9]:
chunk.detectFiducials(generate_masks=True, cameras=chunk.cameras)

print("Found the following fiducials in the images: ")
for marker in chunk.markers:
    print(marker.label)

DetectFiducials: generate_masks = on, mask_dark_pixels = on, generic_detector = on
Processing camera CA196532V0010
Detecting frame edges
Found good frame.
Detected frame edges in 3.84895 seconds
Detecting background
Detected in 2.02001 seconds
Detecting fiducials 
Detected 4 fiducials in 0.896035 seconds
Processing camera CA196532V0011
Detecting frame edges
Found good frame.
Detected frame edges in 2.09626 seconds
Detecting background
Detected in 2.03667 seconds
Detecting fiducials 
Detected 4 fiducials in 0.807657 seconds
Found the following fiducials in the images: 
__auto_0
__auto_1
__auto_2
__auto_3
__auto_4
__auto_5
__auto_6
__auto_7


In [ ]:
# save masks

# define the mask folder and create it if it does not exist
mask_folder = os.path.join(project_fld, "masks")
if not os.path.exists(mask_folder):
    os.makedirs(mask_folder)

# iterate over the cameras
for camera in chunk.cameras:

    # save the mask
    mask_path = os.path.join(mask_folder, f"{camera.label}_mask.tif")
    camera.mask.image().save(mask_path)

In [ ]:
import src.sfm_agi.snippets.create_text_mask as ctm
import numpy as np

for camera in chunk.cameras:
    if camera.enabled and camera.mask:
        mask = camera.mask.image()

        m_width = mask.width
        m_height = mask.height

        # convert to np array
        mask_bytes = mask.tostring()
        existing_mask = np.frombuffer(mask_bytes, dtype=np.uint8).reshape((m_height, m_width))

        # get the image id
        image_id = camera.label.split("_")[0]

        # create a text mask
        import numpy as np
        text_mask = ctm.create_text_mask(mask.width, mask.height, image_id)
        text_mask = text_mask.astype(np.uint8)

        new_mask = np.minimum(existing_mask, text_mask)
        new_mask = Metashape.Image.fromstring(new_mask, new_mask.shape[1], new_mask.shape[0], ' ', datatype='U8')
        mask_obj = Metashape.Mask()
        mask_obj.setImage(new_mask)
        #mask2.setImage(metashape_image)

        camera.mask = mask_obj


In [ ]:
# define the mask folder and create it if it does not exist
mask_folder = os.path.join(project_fld, "masks")
if not os.path.exists(mask_folder):
    os.makedirs(mask_folder)

# iterate over the cameras
for camera in chunk.cameras:

    # save the mask
    mask_path = os.path.join(mask_folder, f"{camera.label}_mask_adapted.tif")
    camera.mask.image().save(mask_path)


In [ ]:
# match photos
chunk.matchPhotos(generic_preselection=True, reference_preselection=False, filter_mask=True)

In [ ]:
chunk.alignCameras()

In [ ]:
def _save_tps(chunk):


    import numpy as np
    import src.display.display_images as di
    import src.load.load_image as li

    # Create a dictionary to map camera labels to file paths
    camera_file_paths = {camera.label: camera.photo.path for camera in chunk.cameras}

    # Initialize a list to store matches
    matches = []

    # Iterate through the camera pairs
    for i, camera1 in enumerate(chunk.cameras):
        for j, camera2 in enumerate(chunk.cameras):
            if i < j:  # Ensure unique pairs
                matches.append(chunk.point_cloud.get_matches(camera1, camera2))

    matches_np = np.array([match.points for match_pair in matches for match in match_pair])

    print(matches_np)

    for (camera1_label, camera2_label), points in tie_points_dict.items():
        image1_path = camera_file_paths[camera1_label]
        image2_path = camera_file_paths[camera2_label]
        image1 = li.load_image(image1_path)
        image2 = li.load_image(image2_path)
        print(f"Tie points between {camera1_label} and {camera2_label}:")
        di.display_images([image1, image2], tie_points=points)

_save_tps(chunk)

In [ ]:
# match photos
chunk.matchPhotos(generic_preselection=True, reference_preselection=False)

_save_tps(chunk)
